## Análise a fim de orientar o tipo do próximo filme a ser desenvolvido em Hollywood

ML com aprendizagem em regressão, pois analisaremos dados passados e atuais, para decisão futura.

## Importar Bibliotecas e Carregar os Dados

In [4]:
import pandas as pd



In [5]:
series = pd.read_csv("./series_imdb.csv")

series.head()

,Unnamed: 0,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,1,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
1,2,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
2,3,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
3,4,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"
4,5,The Lord of the Rings: The Return of the King,2003,U,201 min,"Action, Adventure, Drama",8.9,Gandalf and Aragorn lead the World of Men agai...,94.0,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,1642758,"377,845,905"


## Análise dos dados

In [8]:
series.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     999 non-null    int64  
 1   Series_Title   999 non-null    object 
 2   Released_Year  999 non-null    object 
 3   Certificate    898 non-null    object 
 4   Runtime        999 non-null    object 
 5   Genre          999 non-null    object 
 6   IMDB_Rating    999 non-null    float64
 7   Overview       999 non-null    object 
 8   Meta_score     842 non-null    float64
 9   Director       999 non-null    object 
 10  Star1          999 non-null    object 
 11  Star2          999 non-null    object 
 12  Star3          999 non-null    object 
 13  Star4          999 non-null    object 
 14  No_of_Votes    999 non-null    int64  
 15  Gross          830 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 125.0+ KB


É possível ver que o Runtime, Released_year, Gross estão sendo carregados com formato não correspondente. Runtime deveria ser int64 mas está como object, provavelmente por incluir "min" de minutos, Released_year como são apenas números deveria ser carregado, também, como int64 e, por fim, Gross, em pesquisa, é o faturamento apenas no EUA e Canadá, portanto é um valor que representa dólares, assim deveria ser carregado como int64 por não considerar flutuação de centavos. 

In [12]:
series.isnull().sum()

Unnamed: 0         0
Series_Title       0
Released_Year      0
Certificate      101
Runtime            0
Genre              0
IMDB_Rating        0
Overview           0
Meta_score       157
Director           0
Star1              0
Star2              0
Star3              0
Star4              0
No_of_Votes        0
Gross            169
dtype: int64

Tabelas como Unnamed (ID), Series_Title e Overview possuem baixa influência, portanto não necessárias. Series_title é apenas o título do filme e Overview se torna inútil com a existência da coluna Genre.

Além disso, existem poucas linhas e colunas com valores nulos, portanto é possível limpá-los.

In [13]:
series['Certificate'].value_counts()

Certificate
U           234
A           196
UA          175
R           146
PG-13        43
PG           37
Passed       34
G            12
Approved     11
TV-PG         3
GP            2
TV-14         1
Unrated       1
TV-MA         1
16            1
U/A           1
Name: count, dtype: int64

É possível destacar a falta de especifidade devido a diferença de certificações por países, como é possível destacar em artigo publicado pelo IMDB - https://help.imdb.com/article/contribution/titles/certificates/GU757M8ZJ9ZPXB39?ref_=helpms_helpart_inline&utm_source=chatgpt.com#canada

Issues List
- 


## Data Cleaning

A fim de alcançar maior nível de profundidade na análise, é necessário adicionar uma coluna de severidade para a certificação, que ocorre de forma diferente para cada país mas possui a mesma lógica. Com isso, é possível criar uma coluna com o nível de severidade em escala de 0-10, abrangindo categorias como 12 e 12+ com severidade igual a 3 e 4 (exemplo) e equivalências entre países.